# Исследование надежности заемщиков.

Заказчик: кредитный отдел банка. 

Задача: разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.

Входные данные от банка: статистика о платёжеспособности клиентов. 

Результаты исследования будут учтены при построении модели кредитного скоринга -- специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Обзор данных

Импортируем необходимые библиотеки.

In [1]:
import pandas as pd

Для обзора данных воспользуемся методами head() и info(). 

In [2]:
data = pd.read_csv('data.csv')
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод**

В данных встречается ряд проблем:

* В столбцах days_employed и total_income есть пропущенные значения;
* В столбце days_employed присутствуют отрицательные значения, хотя такого не может быть;
* Некорректно указан тип данных в столбце days_employed;
* Данные в столбце education имеют разный регистр;
* Также стоит проверить на наличие дубликатов и нереалистично больших/маленьких значений.

In [5]:
data.head(3)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья


### Шаг 2. Предобработка данных

### Шаг 2.1 Проверка данных на аномалии.

Ранее мы уже обратили внимание, что в столбце days_employed встречаются отрицательные значения. Исправим это. 

In [6]:
data['days_employed'] = abs(data['days_employed'])
data['days_employed'].describe()

count     19351.000000
mean      66914.728907
std      139030.880527
min          24.141633
25%         927.009265
50%        2194.220567
75%        5537.882441
max      401755.400475
Name: days_employed, dtype: float64

В столбце встречаются аномально большие значения (401755 дней это 1100 лет). Исправим это чуть позже.

Проверим остальные столбцы на наличие аномалий.

**children** -- количество детей в семье

In [7]:
data['children'].value_counts().sort_index()

-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64

В столбце children встречаются отрицательные значения. Исправим это. 

In [8]:
data['children'] = abs(data['children'])
data['children'].value_counts().sort_index()

0     14149
1      4865
2      2055
3       330
4        41
5         9
20       76
Name: children, dtype: int64

Из данных видно, что встречаются заемщики с 20-ю детьми. Проверим, возможно ли это. Выведем список этих заемщиков и посмотрим на их возраст.

In [9]:
data[data['children'] == 20].groupby('dob_years').size()

dob_years
0     1
21    1
23    1
24    1
25    1
26    1
27    2
29    2
30    3
31    2
32    2
33    2
34    3
35    2
36    2
37    4
38    1
39    1
40    4
41    2
42    3
43    2
44    2
45    3
46    3
48    1
49    3
50    3
51    1
52    1
53    1
54    1
55    1
56    5
57    1
59    2
60    1
61    1
62    1
64    1
69    1
dtype: int64

Часть заемщиков — это молодые люди, которые не могут иметь такое количество детей. Делаем вывод, что была допущена ошибка и вместо значения 2 написали 20. Внесем изменения.

In [10]:
data.loc[(data.children == 20), 'children'] = 2
data['children'].value_counts().sort_index()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

**dob_years** -- возраст клиента в годах

In [11]:
data['dob_years'].describe()

count    21525.000000
mean        43.293380
std         12.574584
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

В столбце dob_years встречается значение 0. Возраст клиента в годах не может быть равным нулю. Посчитаем долю таких заемщиков.

In [12]:
round((data['dob_years'] == 0).sum()/data['dob_years'].count(),3)

0.005

Возраст 0 встречается реже, чем в 1% случаев. Такими данными можно пренебречь. Для этого уберем эти строки из датафрейма. 

In [13]:
data = data[data['dob_years'] != 0]

**education** -- уровень образования клиента

In [14]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

В столбце education встречаются одни и те же категории, но с разными регистрами. Приведем все значения к нижнему регистру. 

In [15]:
data['education'] = data['education'].str.lower()
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

**education_id** -- идентификатор уровня образования

In [16]:
data['education_id'].unique()

array([0, 1, 2, 3, 4], dtype=int64)

В столбце education_id нет аномалий.

**family_status** -- семейное положение

In [17]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

В столбце family_status нет аномалий. 

**family_status_id** -- идентификатор семейного положения

In [18]:
data['family_status_id'].unique()

array([0, 1, 2, 3, 4], dtype=int64)

В столбце family_status_id нет аномалий.

**gender** -- пол клиента

In [19]:
data['gender'].value_counts().sort_values()

XNA        1
M       7259
F      14164
Name: gender, dtype: int64

В колонке gender встречается значение XNA. Такое значение только одно, оно не критично для нашей задачи. Избавимся от нее.

In [20]:
data = data[data['gender'] != 'XNA']

**income_type** -- тип занятости

In [21]:
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

В столбце income_type нет аномалий.

**debt** -- имел ли задолженность по возврату кредитов

In [22]:
data['debt'].unique()

array([0, 1], dtype=int64)

В столбце debt нет аномалий.

**total_income** --  ежемесячный доход

In [23]:
data['total_income'].describe()

count    1.925900e+04
mean     1.674633e+05
std      1.030897e+05
min      2.066726e+04
25%      1.030929e+05
50%      1.450055e+05
75%      2.033503e+05
max      2.265604e+06
Name: total_income, dtype: float64

В столбце total_income нет аномалий.

**purpose**

In [24]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

В столбце purpose встречаются выражения с одинаковым значением. Например, "свадьба" и "сыграть свадьбу". Исправим их позже.

### Шаг 2.2 Заполнение пропусков.

Пропущенные значения встречаются в столбцах days_employed и total_income.

Выясним количество пропусков в столбце days_employed и какую долю они составляют. 

In [25]:
data['days_employed'].isna().sum()

2164

In [26]:
round(data['days_employed'].isna().sum() / data['days_employed'].count(),2)

0.11

Пропущенные значения в столбце days_employed составляют 11%.

Проведем те же манипуляции со столбцом total_income.

In [27]:
data['total_income'].isna().sum()

2164

In [28]:
round(data['total_income'].isna().sum() / data['total_income'].count(),2)

0.11

Пропущенные значения в столбце total_income составляют 11%.

Количество пропусков в days_employed и total_income совпадают. Чтобы избавиться от пропусков, заполним их медианными значениями. Медианное значение является более приоритетным по сравнению со средним, так как в столбцах есть значения, оторванные от всей основной массы. Однако заполнить все пропуски одним общим медианным значением нельзя, так как в наших данных присутствуют заемщики разных возрастов, с разным уровнем образованием и т.д.

Для определения медианного значения в колонке days_employed разделим пользователей по уровню их образования. Так мы узнаем какие медианное значение трудового стажа соответствует каждому из видов образования.

In [29]:
# медианное значение трудового стажа по уровням образования

median_days_employed = round(data.groupby(['education_id'])['days_employed'].median(), 2)
median_days_employed

education_id
0    1895.75
1    2392.81
2    1209.18
3    3043.93
4    5660.06
Name: days_employed, dtype: float64

In [30]:
# заполнение пропусков 

data.loc[(data['days_employed'].isna()) & (data['education_id'] == 0), 'days_employed'] = median_days_employed[0]
data.loc[(data['days_employed'].isna()) & (data['education_id'] == 1), 'days_employed'] = median_days_employed[1]
data.loc[(data['days_employed'].isna()) & (data['education_id'] == 2), 'days_employed'] = median_days_employed[2]
data.loc[(data['days_employed'].isna()) & (data['education_id'] == 3), 'days_employed'] = median_days_employed[3]
data.loc[(data['days_employed'].isna()) & (data['education_id'] == 4), 'days_employed'] = median_days_employed[4]

Теперь избавимся от пропусков в колонке total_income. В этом случае мы разделим заемщиков по их типам (студент, пенсионер и т.д.) и рассчитаем медианный уровень ежемесячного дохода для каждого из них.

In [31]:
# медианное значение ежемесячного дохода по типу заемщика

median_total_income = round(data.groupby(['income_type'])['total_income'].median(), 2)
median_total_income

income_type
безработный        131339.75
в декрете           53829.13
госслужащий        150521.92
компаньон          172268.93
пенсионер          118557.18
предприниматель    499163.14
сотрудник          142594.40
студент             98201.63
Name: total_income, dtype: float64

In [32]:
# заполнение пропусков 

data.loc[(data['total_income'].isna()) & (data['income_type'] == 'безработный'), 'total_income'] = median_total_income[0]
data.loc[(data['total_income'].isna()) & (data['income_type'] == 'в декрете'), 'total_income'] = median_total_income[1]
data.loc[(data['total_income'].isna()) & (data['income_type'] == 'госслужащий'), 'total_income'] = median_total_income[2]
data.loc[(data['total_income'].isna()) & (data['income_type'] == 'компаньон'), 'total_income'] = median_total_income[3]
data.loc[(data['total_income'].isna()) & (data['income_type'] == 'пенсионер'), 'total_income'] = median_total_income[4]
data.loc[(data['total_income'].isna()) & (data['income_type'] == 'предприниматель'), 'total_income'] = median_total_income[5]
data.loc[(data['total_income'].isna()) & (data['income_type'] == 'сотрудник'), 'total_income'] = median_total_income[6]
data.loc[(data['total_income'].isna()) & (data['income_type'] == 'студент'), 'total_income'] = median_total_income[7]

Проверим, остались ли пропущенные значения.

In [33]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21423 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21423 non-null  int64  
 1   days_employed     21423 non-null  float64
 2   dob_years         21423 non-null  int64  
 3   education         21423 non-null  object 
 4   education_id      21423 non-null  int64  
 5   family_status     21423 non-null  object 
 6   family_status_id  21423 non-null  int64  
 7   gender            21423 non-null  object 
 8   income_type       21423 non-null  object 
 9   debt              21423 non-null  int64  
 10  total_income      21423 non-null  float64
 11  purpose           21423 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


### Шаг 2.3. Изменение типов данных.

Как мы определили ранее, в столбце days_employed неверно указан тип данных. 
Также стоит заменить типа данных и в столбце total_income. Это никак не повлияет на результат, но работать со столбцом станет удобнее. 

In [34]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

### Шаг 2.4. Удаление дубликатов.

Определим есть ли в данных дубликаты, выясним их количество и их долю.

In [35]:
data.duplicated().sum()

71

In [36]:
round(data.duplicated().sum() / data['purpose'].count(), 3)

0.003

Доля дубликатов составляет менее 1%. Принимаем решение удалить их. 

In [37]:
data = data.drop_duplicates(keep='first')

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создадим 2 новых датафрейма (1. каждому уникальному значению из education соответствует уникальное значение education_id — в первом, 2. каждому уникальному значению из family_status соответствует уникальное значение family_status_id — во втором). И удалим из исходного датафрейма колонки education и family_status. Новые датафреймы — это словари, к которым мы сможем обращаться по идентификатору.

In [38]:
# словарь 1

education_ref = data[['education', 'education_id']] 
education_ref = education_ref.drop_duplicates().reset_index(drop=True)
education_ref 

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [39]:
# словарь 2

family_status_ref = data[['family_status', 'family_status_id']]
family_status_ref = family_status_ref.drop_duplicates().reset_index(drop=True)
family_status_ref

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


Удалим из исходного датафрейма 2 колонки education и family_status.

In [40]:
data.drop(columns = ['education'],axis = 1, inplace=True)
data.drop(columns = ['family_status'],axis = 1, inplace=True)

### Шаг 2.6. Категоризация дохода.

Добавим к данным новый столбец с категориями по уровню ежемесячного дохода.

In [41]:
def total_income_category(income):
    if 0<= income <= 30000:
        return 'Категория E'
    if 30000 < income <= 50000:
        return 'Категория D'
    if 50000 < income <= 200000:
        return 'Категория C'
    if 200000 < income <= 1000000:
        return 'Категория B'
    return 'Категория A'

data['total_income_category'] = data['total_income'].apply(total_income_category)

In [42]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21352 entries, 0 to 21524
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   children               21352 non-null  int64 
 1   days_employed          21352 non-null  int32 
 2   dob_years              21352 non-null  int64 
 3   education_id           21352 non-null  int64 
 4   family_status_id       21352 non-null  int64 
 5   gender                 21352 non-null  object
 6   income_type            21352 non-null  object
 7   debt                   21352 non-null  int64 
 8   total_income           21352 non-null  int32 
 9   purpose                21352 non-null  object
 10  total_income_category  21352 non-null  object
dtypes: int32(2), int64(5), object(4)
memory usage: 1.8+ MB


### Шаг 2.7. Категоризация целей кредита.

Напишем функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category со следующими категориями:
* "операции с автомобилем",
* "операции с недвижимостью",
* "проведение свадьбы",
* "получение образования" 

In [43]:
def total_purpose(purpose):
    if 'авто' in purpose:
        return 'операции с автомобилем'
    if 'образован' in purpose:
        return 'получение образования'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'жиль' or 'недвиж':
        return 'операции с недвижимостью'
    return 'прочее'
data['purpose_category'] = data['purpose'].apply(total_purpose)
data.head(3)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,Категория B,операции с недвижимостью
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,Категория C,операции с автомобилем
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,Категория C,операции с недвижимостью


Проверим что получилось. 

In [44]:
data['purpose_category'].value_counts()

операции с недвижимостью    10763
операции с автомобилем       4284
получение образования        3995
проведение свадьбы           2310
Name: purpose_category, dtype: int64

### Шаг 3. Анализ данных

### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [45]:
data.groupby('children')['debt'].agg({'count','sum', 'mean'})

,count,mean,sum
children,,,
0,14021,0.075458,1058
1,4839,0.091341,442
2,2114,0.095553,202
3,328,0.082317,27
4,41,0.097561,4
5,9,0.000000,0


**Вывод**

Из полученных данных можно сделать вывод, что для случаев, когда выборка достаточно большая (0,1,2 ребенка), зависимость есть. Чем больше количество детей, тем больше процент просроченных задолженностей. 

Зависимость не сохраняется для людей, имеющих 3 либо 5 детей. В связи с этим делаем вывод, что для этих категорий возможно нужно больше информации. 

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [46]:
group_family_status = data.groupby(
    ['family_status_id']).agg({'debt': ['count', 'sum']}).merge(family_status_ref, on='family_status_id', how='left')
group_family_status[('debt', 'sum')]/group_family_status[('debt', 'count')]
group_family_status = group_family_status.rename(columns={('debt', 'count'): 'count', ('debt', 'sum'): 'sum'})
group_family_status['percent'] = round(group_family_status['sum'] / group_family_status['count'] * 100, 2)
group_family_status

C:\Users\GIGABYTE\AppData\Local\Temp\ipykernel_2224\608250004.py:1: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left, 1 on the right)
  group_family_status = data.groupby(


,family_status_id,count,sum,family_status,percent
0,0,12290,927,женат / замужем,7.54
1,1,4129,386,гражданский брак,9.35
2,2,954,62,вдовец / вдова,6.50
3,3,1185,85,в разводе,7.17
4,4,2794,273,Не женат / не замужем,9.77


**Вывод**

Из данных выше можно сделать вывод, что зависимость есть. Категории "Не женат/не замужем" и "гражданский брак", то есть люди, не бывавшие в браке, имеют больший процент невозвратов в срок. Те, кто развелись или овдовели чаще платят в срок.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [47]:
debt_and_total_income = data.pivot_table(index=['total_income_category'], values='debt', aggfunc=['count', 'sum', 'mean'])
debt_and_total_income.style.format({('mean', 'debt'): '{:.2%}'})

,count,sum,mean
,debt,debt,debt
total_income_category,,,
Категория A,25,2,8.00%
Категория B,5015,355,7.08%
Категория C,15942,1353,8.49%
Категория D,348,21,6.03%
Категория E,22,2,9.09%


**Вывод**

Из результата выше видно, что зависимости между уровнем дохода и возвратом кредита в срок нет. Зависимость сложно уловить еще по причине того, что в нескольких категориях очень мало заемщиков. 

### Как разные цели кредита влияют на его возврат в срок?

In [48]:
debt_purpose = data.groupby('purpose_category').agg({'debt': ['count', 'sum']})
debt_purpose.columns = ['count', 'sum']
debt_purpose['percent'] = round(debt_purpose['sum'] / debt_purpose['count'] * 100, 2)
debt_purpose.sort_values(by='percent', ascending=False)

,count,sum,percent
purpose_category,,,
операции с автомобилем,4284,400,9.34
получение образования,3995,370,9.26
проведение свадьбы,2310,184,7.97
операции с недвижимостью,10763,779,7.24


**Вывод**

Как видно из таблицы выше, чаще всего задолженности по кредиту бывают у тех, кто берет кредит для операций с автомобилем и получением образования, а реже всего -- у новобрачных и тех кто берет кредит на операции с недвижимостью. 

## Общий вывод:

Переданные данные не были готовы к анализу, в них встречался ряд проблем: пропущенные значения, некорректные типы данных, дубликаты, нереалистично большие/маленькие значения и т.д. Была проведена тщательная предобработка данных. 

Для выявления критериев, влияющих на возврат кредита в срок, были применены сводные таблицы и группировки. 

Главная задача исследования, разобраться влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Из проведенной работы можно сделать выводы:
* Семейное положение влияет на факт погашения кредита в срок. Люди не в браке и не бывавшие в браке имеют больший процент невозвратов в срок. Те, кто развелись или овдовели чаще платят в срок.
* Количество детей может влиять на факт погашения кредита в срок (это видно для семей с 0/1/2 детьми), но данных по другим категориям недостаточно, чтобы сделать однозначный вывод. Чем больше количество детей, тем больше количества просроченных задолженностей.

Возникшие аномалии и проблемы с данными могут быть связаны с техническими ошибками и человеческим фактором. 
